<a href="https://colab.research.google.com/github/sidfeiner/EpitopeWorkshop/blob/master/EpitopeWorkshop-Sid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
token = getpass('insert token')
!git clone https://$token@github.com/sidfeiner/EpitopeWorkshop.git
!mv EpitopeWorkshop workshop

insert token··········
Cloning into 'EpitopeWorkshop'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 224 (delta 116), reused 163 (delta 59), pack-reused 0
Receiving objects: 100% (224/224), 10.61 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [2]:
!cd workshop/ && pip3 install -e .
import sys
sys.path.append('/content/workshop/')
!pip install fsspec

Obtaining file:///content/workshop
     |████████████████████████████████| 2.3 MB 4.7 MB/s 
     |████████████████████████████████| 135 kB 85.0 MB/s 
     |████████████████████████████████| 87 kB 8.7 MB/s 
     |████████████████████████████████| 41 kB 254 kB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=360b810636acbdff84250747b2528df13aad8dc0adb06d178cdbdc2f0c976cae
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50466 sha256=03768bf5b54992bb3358baa180aae1908b6ee4146dee0d6c983594db9567f239
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995dbf11db6cede602e7ef
  Created wheel for quantiprot: filename=quantiprot-0.2.4-py3-none-any.whl size=151137 sha256=760d3f2c214ae3aa94bc4c7a48ea419e78b8400ddb6f6ac77f22421acf615d55
  Stored in directory: /root/.cache/pip/wheels/93/8d/6

In [3]:
from EpitopeWorkshop.process import read
from EpitopeWorkshop.common.conf import DEFAULT_WINDOW_SIZE, DEFAULT_WITH_SLIDING_WINDOW
from EpitopeWorkshop.process.features import FeatureCalculator
import dask.dataframe as dd

sequences_file_path = '/content/workshop/data/iedb_linear_epitopes.fasta'
window_size = 9
partitions_amt=2

df = read.load_fasta_fasta(sequences_file_path, True, window_size, 1)
ddf = dd.from_pandas(df, npartitions=partitions_amt)
calculator = FeatureCalculator()
df = calculator.calculate_features(ddf)

In [4]:
from EpitopeWorkshop.common import utils
import torch
import torchtext
from EpitopeWorkshop.common import contract
from EpitopeWorkshop.dataset.EpitopeDataset import EpitopeDataset

sub_df = df[contract.NETWORK_LABELED_INPUT_ARGS]
labels_df = df[contract.IS_IN_EPITOPE_COL_NAME]
ds = EpitopeDataset(sub_df)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ds_train, ds_valid, ds_test = ds.splits()
print(len(ds_train), len(ds_valid), len(ds_test))

3691 552 1049


In [5]:
from torchtext import data
BATCH_SIZE = 25
dl_train, dl_valid, dl_test = ds.iters(batch_size=32)

x0, y0 = ds[0]

In [6]:
from EpitopeWorkshop.cnn.cnn import CNN

model = CNN(len(contract.NETWORK_INPUT_ARGS))

print('x0 shape=', x0.shape, end='\n\n')
print('LeNet(x0)=', model(x0), end='\n\n')
print('shape=', net(x0).shape)

x0 shape= torch.Size([26])



RuntimeError: ignored